Here till now our query has done single forward pass, given the query call the right tool with right parameters and get back the response.

what if we ask complex question consisting of multiple steps,or vague question that needs clarification?

Instead of tool calling in single shot setting, agent is able to reason over tools and multiple steps. 

We will use function calling agent implementation, which is agent that natively integrates with the function calling capabilities of LLMs.



In [1]:
import os
from dotenv import load_dotenv


In [3]:
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

python-dotenv could not parse statement starting at line 1
python-dotenv could not parse statement starting at line 8


In [4]:
import nest_asyncio
nest_asyncio.apply()

In [5]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(input_files = ["/Users/apple/Desktop/project/LLM/agentic_RAG/Data/metagpt.pdf"]).load_data()

In [8]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

In [10]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding


Settings.llm=OpenAI(model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model = "text-embedding-ada-002")

Index tools

In [12]:
from llama_index.core import SummaryIndex, VectorStoreIndex

summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(nodes)

Query Engine

In [13]:
summary_query_engine = summary_index.as_query_engine(
    response_mode = "tree_summarize",
    use_async = True
)
vector_query_engine = vector_index.as_query_engine()

convert engine to query tools

In [22]:
from llama_index.core.tools import QueryEngineTool

summary_tool= QueryEngineTool.from_defaults(
    name="summary_tool",
    query_engine = summary_query_engine,
    description = (
        "Useful for summarization questions related to MetaGPT"
    ),
)

vector_tool= QueryEngineTool.from_defaults(
    name="vector_tool",
    query_engine = vector_query_engine,
    description = (
        "useful for retrieving specific context from the MetaGPT paper."
    ),
)


Agent

1. AgentRunner
2. AgentWorker

Agent Runner:- overall task dispatcher, which is responsible for creating a task, orchestrating runs of agent worker on top of a given task,

and being able to return back the final response to the user.

Agent Task Orchestrator

AgentState: mapping from task_id to taskstate

Taskstate:-

1. Task
2. Completed steps
3. step queue

Memory: Conversation memory

AgentWorker:- Executing the next step of a given agent

Task Reasoning and Execution:- 

Tools: 

1. Vector Tool
2. Summary Tool

LLM: LLM to use

In [23]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model = "gpt-3.5-turbo",temperature=0)

In [24]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_tool, summary_tool],
    llm=llm,
    verbose=True
)
agent = AgentRunner(agent_worker)

In [25]:
response = agent.query(
    "Tell me about the agent roles in MetaGPT,"
    "and then how they communicate with each other."
)

Added user message to memory: Tell me about the agent roles in MetaGPT,and then how they communicate with each other.
=== Calling Function ===
Calling function: summary_tool with args: {"input": "Agent roles in MetaGPT"}
=== Function Output ===
The agent roles in MetaGPT are the Product Manager, Architect, Project Manager, Engineer, and QA Engineer. Each role has specific responsibilities and contributes specialized outputs to the collaborative software development process. The Product Manager focuses on requirements analysis and documentation, the Architect designs the technical specifications and system architecture, the Project Manager allocates tasks and oversees project execution, the Engineer implements the code based on the technical specifications, and the QA Engineer formulates test cases to ensure code quality. These roles work together in a structured workflow to efficiently develop software solutions.
=== Calling Function ===
Calling function: summary_tool with args: {"inpu

When you run multi step query like this, you want to make sure that your are actually able to trace the sources. 

We are able to look at response that source nodes.

In [26]:
print(response.source_nodes[0].get_content(metadata_mode="all"))

page_label: 1
file_name: metagpt.pdf
file_path: /Users/apple/Desktop/project/LLM/agentic_RAG/Data/metagpt.pdf
file_type: application/pdf
file_size: 16911937
creation_date: 2025-05-31
last_modified_date: 2025-05-31

Preprint
METAGPT: M ETA PROGRAMMING FOR A
MULTI -AGENT COLLABORATIVE FRAMEWORK
Sirui Hong1∗, Mingchen Zhuge2∗, Jonathan Chen1, Xiawu Zheng3, Yuheng Cheng4,
Ceyao Zhang4, Jinlin Wang1, Zili Wang, Steven Ka Shing Yau5, Zijuan Lin4,
Liyang Zhou6, Chenyu Ran1, Lingfeng Xiao1,7, Chenglin Wu1†, J¨urgen Schmidhuber2,8
1DeepWisdom, 2AI Initiative, King Abdullah University of Science and Technology,
3Xiamen University, 4The Chinese University of Hong Kong, Shenzhen,
5Nanjing University, 6University of Pennsylvania,
7University of California, Berkeley, 8The Swiss AI Lab IDSIA/USI/SUPSI
ABSTRACT
Remarkable progress has been made on automated problem solving through so-
cieties of agents based on large language models (LLMs). Existing LLM-based
multi-agent systems can already solve simp

Full agent Reasoning loop

|<-----------------------------------------------------------------------------------|
Query --->   Agent Runner --- > Agent Worker                     |  --> Query Engine | ----> Response
       |     Agent Runner < --- Agent Worker    ----> Tools ---->|  --> Query Engine |
       |     Reasoning                                                               |
       |     & planning layer                                                        |
       |-----------------------------------> memory <--------------------------------|

calling agent query allows you to query the agent in a one off manner, but does not preserve state.

lets try maintaining conversation history over time.

Agent are able to maintain chats in a conversational memory buffer.  

The memory module can be customized, but by default it's a flat list of items that's a rolling buffer depending on the size of the context window of the LLM.

Therefore, When the agent decides to use a tool and not only uses a current chat, but also the previous conversation history to take the next step or perform

the next action.

In [27]:
response = agent.chat(
    "Tell me about the evaluation datasets used."
)

Added user message to memory: Tell me about the evaluation datasets used.
=== Calling Function ===
Calling function: summary_tool with args: {"input": "Evaluation datasets used in MetaGPT"}
=== Function Output ===
The evaluation datasets used in MetaGPT include HumanEval, MBPP, and a self-generated SoftwareDev dataset. HumanEval consists of 164 handwritten programming tasks, while MBPP comprises 427 Python tasks covering core concepts and standard library features. The SoftwareDev dataset includes 70 representative examples of software development tasks with diverse scopes, such as mini-games, image processing algorithms, and data visualization. These datasets were utilized to evaluate the performance of MetaGPT in code generation tasks.
=== LLM Response ===
The evaluation datasets used in MetaGPT include HumanEval, MBPP, and a self-generated SoftwareDev dataset. HumanEval consists of 164 handwritten programming tasks, while MBPP comprises 427 Python tasks covering core concepts and st

In [28]:
response = agent.chat(
    "Tell me the overall results over one of the above datasets"
)

Added user message to memory: Tell me the overall results over one of the above datasets
=== Calling Function ===
Calling function: summary_tool with args: {"input": "Overall results of MetaGPT on the HumanEval dataset"}
=== Function Output ===
MetaGPT demonstrated superior performance on the HumanEval dataset, surpassing previous methods in terms of functional accuracy, task completion rate, and code quality. Its collaborative framework, which includes role specialization, structured communication, and executable feedback mechanisms, played a significant role in generating high-quality code solutions. Additionally, MetaGPT achieved an average score of 0.724 in the HumanEval benchmark, outperforming GPT-4-0613, which was more responsive to prompts, code parsing, and post-processing outcomes compared to GPT-3.5-Turbo-0613. GPT-4 consistently outperformed GPT-3.5-Turbo-0613 in various settings during the HumanEval experiments.
=== LLM Response ===
MetaGPT demonstrated superior performanc

Agent Control


Key Benefits:

1. Decoupling of Task Creation and Execution:

User gain the flexibility to schedule task execution according to their needs.

2. Enhanced Debuggability: 

Offers deeper insights into each step of execution process, improving troubleshooting capabilities.

3. Steerability:

Allows users to directly modify intermediate steps and incorporate human feedback for refined control.

Having this low level agent interface is powerful for two main reasons.

1. Debuggability :- Transparency and visibility into what's actually going on under the hood.

Especially, say if you agent isn't working the first time around, then you can actually go in and trace the execution of the agent, 

see where its failing and actually try out different inputs to see if that actually modifies the agent execution into a correct response.

2. Richer UXs, where you are building a product experiences around this core agent capability.

Example :- You are building a product experience around this core agent capability. Lets say you want to listen to human feedback

in the middle of agent execution, as opposed to only after the agent execution is complete for a given task. Then, you can imagine creating some sort of 

async queue, where you are able to listen to inputs from humans throughout the middle of agent execution.

And if human input actually comes in, you can actually interrupt and modify the execution of an agent its going on through a larger task,

as opposed to having to wait until the agent task is complete.

In [29]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_tool, summary_tool],
    llm = llm,
    verbose = True
)

agent = AgentRunner(agent_worker)

In [30]:
task = agent.create_task(
    "Tell me about the agent roles in MetaGPT,"
    "and then how they communicate with each other."
)

In [31]:
step_output = agent.run_step(task.task_id)

Added user message to memory: Tell me about the agent roles in MetaGPT,and then how they communicate with each other.
=== Calling Function ===
Calling function: vector_tool with args: {"input": "Agent roles in MetaGPT"}
=== Function Output ===
In MetaGPT, the agent roles include Product Manager, Architect, Project Manager, Engineer, and QA Engineer. Each role has specific responsibilities and skills tailored to their tasks within the software company simulated by MetaGPT. The roles are designed to collaborate effectively, with each agent contributing specialized outputs to solve complex problems and tasks.
=== Calling Function ===
Calling function: vector_tool with args: {"input": "How do agents communicate with each other in MetaGPT?"}
=== Function Output ===
Agents in MetaGPT communicate with each other by utilizing a shared message pool where they publish structured messages and subscribe to relevant messages based on their profiles. This communication protocol enhances role communi

In [32]:
completed_steps = agent.get_completed_steps(task.task_id)
print(f"Num completed for task {task.task_id}: {len(completed_steps)}")
print(completed_steps[0].output.sources[0].raw_output)

Num completed for task 9fd7740d-5109-43d5-9276-ad6ccb00681d: 1
In MetaGPT, the agent roles include Product Manager, Architect, Project Manager, Engineer, and QA Engineer. Each role has specific responsibilities and skills tailored to their tasks within the software company simulated by MetaGPT. The roles are designed to collaborate effectively, with each agent contributing specialized outputs to solve complex problems and tasks.


In [36]:
upcoming_steps = agent.get_upcoming_steps(task.task_id)
print(f"Num upcoming steps for task{task.task_id}: {len(upcoming_steps)}")
upcoming_steps[0]

Num upcoming steps for task9fd7740d-5109-43d5-9276-ad6ccb00681d: 1


TaskStep(task_id='9fd7740d-5109-43d5-9276-ad6ccb00681d', step_id='e7292cb7-2756-4097-b793-9d50906c4855', input=None, step_state={}, next_steps={}, prev_steps={}, is_ready=True)

input is none because the way agent works is actually just auto generates, action from the conversation history and doesn't need to generate an additional external input.

In [39]:
step_output = agent.run_step(
    task.task_id,
    input = "What about how agents share information?"
)

Added user message to memory: What about how agents share information?
=== LLM Response ===
Agents in MetaGPT share information by utilizing a shared message pool. This pool allows agents to publish structured messages and access messages from other agents directly. By storing information in this global message pool, agents can exchange information transparently without the need for one-to-one communication. Additionally, agents can subscribe to specific information based on their role profiles, ensuring they receive only relevant information and avoid information overload.


check if the output is the last steps and need to synthesis the answer

In [41]:
step_output = agent.run_step(task.task_id)
print(step_output.is_last)

IndexError: pop from an empty deque

In [42]:
response = agent.finalize_response(task.task_id)

In [43]:
print(response)

Agents in MetaGPT share information by utilizing a shared message pool. This pool allows agents to publish structured messages and access messages from other agents directly. By storing information in this global message pool, agents can exchange information transparently without the need for one-to-one communication. Additionally, agents can subscribe to specific information based on their role profiles, ensuring they receive only relevant information and avoid information overload.
